In [1]:
import logging

from pathlib import Path

from Pegasus.api import *

logging.basicConfig(level=logging.DEBUG)

#Properties
props = Properties()
props["pegasus.data.configuration"] = "condorio"
props["dagman.retry"] = "2"
props.write()

with open("f.a", "w") as f:
    f.write("Hello world")

fa = File("f.a").add_metadata()

rc = ReplicaCatalog()\
        .add_replica("local", "f.a", Path(".").resolve()  / "f.a")\
        .write()

#transformation
echo =  Transformation(
                "echo",
                site="condorpool",
                pfn="/usr/bin/echo",
                is_stageable=False,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )
tc = TransformationCatalog()\
    .add_transformations(echo)\
    .write()

wf = Workflow("hello-world")
job_echo =  Job(echo)\
        .add_inputs(fa)\
        .set_stdout("out.txt")

wf.add_jobs(job_echo)



In [2]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)

Plan:
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.07.10 03:48:21.262 UTC:    
2020.07.10 03:48:21.267 UTC:   ----------------------------------------------------------------------- 
2020.07.10 03:48:21.273 UTC:   File for submitting this DAG to HTCondor           : hello-world-0.dag.condor.sub 
2020.07.10 03:48:21.278 UTC:   Log of DAGMan debugging messages                 : hello-world-0.dag.dagman.out 
2020.07.10 03:48:21.284 UTC:   Log of HTCondor library output                    

[##################################################] 100.0% ..Success (Completed: 8, Queued: 0, Running: 0, Failed: 0)


Analyzer:
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

 Submit Directory   : /home/scitech/shared-data/workflow/scitech/pegasus/hello-world/run0001
 Total jobs         :      8 (100.00%)
 # jobs succeeded   :      8 (100.00%)
 # jobs failed      :      0 (0.00%)
 # jobs held        :      0 (0.00%)
 # jobs unsubmitted :      0 (0.00%)

 

Statistics:
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#     * Succeeded - total count of succeeded tasks/jobs/sub workflows.
#     * Failed - total count of 